In [1]:
import os
import ipywidgets as widgets
from IPython.display import display
import threading
import time
from PIL import Image, ImageDraw
import io

import numpy as np
import cv2

In [2]:
# HSV閾値のためのスライダーを作成
low_h_slider = widgets.IntSlider(min=0, max=180, value=90, description='Low H')
high_h_slider = widgets.IntSlider(min=0, max=180, value=100, description='High H')
low_s_slider = widgets.IntSlider(min=0, max=255, value=140, description='Low S')
high_s_slider = widgets.IntSlider(min=0, max=255, value=255, description='High S')
low_v_slider = widgets.IntSlider(min=0, max=255, value=0, description='Low V')
high_v_slider = widgets.IntSlider(min=0, max=255, value=255, description='High V')


def parse_filename(filename):
    """ ファイル名からx, y, rを抽出する """
    parts = filename.split('_')
    if len(parts) >= 4:
        x, y, r = map(int, parts[1:4])
        return x, y, r
    return None, None, None

def draw_circle_on_image(image, x, y, r):
    """ 画像に円を描画する """
    img = cv2.circle(image, (x, y), r, (0, 255, 0), 2)
    return image
    
def apply_hsv_threshold(image):
    # low_h, high_h = 90, 100  # Example range for yellow hue
    # low_s, high_s = 140, 255 # Example range for saturation
    # low_v, high_v = 0, 255 # Example range for value

    low_h, high_h = low_h_slider.value, high_h_slider.value
    low_s, high_s = low_s_slider.value, high_s_slider.value
    low_v, high_v = low_v_slider.value, high_v_slider.value
    

    # Convert the image from RGB to HSV
    hsv_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    
    # Define the lower and upper bounds of the HSV threshold
    lower_bound = np.array([low_h, low_s, low_v])
    upper_bound = np.array([high_h, high_s, high_v])
    
    # Create a mask where pixels within the threshold are white, and others are black
    mask = cv2.inRange(hsv_image, lower_bound, upper_bound)
    
    # Create an all black image
    black_image = np.zeros_like(image)
    
    # Copy the pixels from the original image where the mask is white
    result_image = np.where(mask[:, :, None] == 255, image, black_image)
    
    return result_image

In [3]:
def display_images_with_mask(image_directory, interval=0.5):
    # 画像ファイルのリストを取得
    images = [f for f in os.listdir(image_directory) if f.endswith(('.png', '.jpg', '.jpeg', '.gif'))]
    image_path = [os.path.join(image_directory, image) for image in images]
    total_images = len(images)

    # 画像とファイル名、インデックスを表示するためのウィジェットを作成
    image_widget = widgets.Image(
        value=open(image_path[0], 'rb').read() if images else None,
        format='jpg',
        width=224,
        height=224,
    )
    mask_image_widget = widgets.Image(
        format='jpg',
        width=224,
        height=224,
    )

    filename_label = widgets.Label(value=f"{images[0]} - 1/{total_images}" if images else "No images available")

    is_playing = False
    current_image = 0
    # 画像を更新する関数
    def update_image():
        nonlocal current_image
        if images:
            # original_image = Image.open(image_path[current_image])
            original_image = cv2.imread(image_path[current_image])
            mask_image = apply_hsv_threshold(original_image)
            
            x, y, r = parse_filename(images[current_image])
            image_with_circle = draw_circle_on_image(mask_image, x, y, r) if x is not None else open(image_path[current_image], 'rb').read()
            
            _, encoded_image = cv2.imencode('.jpg', image_with_circle)
            mask_image_widget.value = encoded_image.tobytes()
            image_widget.value = open(image_path[current_image], 'rb').read()

            
            filename_label.value = f"{images[current_image]} - {current_image + 1}/{total_images}"


    # 自動再生を行う関数
    def autoplay_images():
        nonlocal is_playing, current_image
        while is_playing and current_image < total_images:
            update_image()
            time.sleep(interval)
            current_image += 1
            if current_image >= total_images:
                current_image = 0

    # 再生・停止・戻る・削除ボタンのイベントハンドラ
    def on_play_clicked(b):
        nonlocal is_playing
        if not is_playing and images:
            is_playing = True
            threading.Thread(target=autoplay_images).start()

    def on_stop_clicked(b):
        nonlocal is_playing
        is_playing = False

    def on_back_clicked(b):
        nonlocal current_image
        if images:
            if current_image > 0:
                current_image -= 1
            else:
                current_image = total_images - 1
            update_image()

    def on_delete_clicked(b):
        nonlocal current_image, total_images
        if images:
            # 削除処理
            os.remove(image_path[current_image])
            del images[current_image]
            del image_path[current_image]
            total_images -= 1
            if current_image >= total_images:
                current_image = total_images - 1
            update_image()

    # 再生・停止・戻る・削除ボタンの作成
    # 再生・停止・戻る・削除ボタンの作成
    play_button = widgets.Button(description="Play")
    stop_button = widgets.Button(description="Stop")
    back_button = widgets.Button(description="Back")
    delete_button = widgets.Button(description="Delete")
    play_button.on_click(on_play_clicked)
    stop_button.on_click(on_stop_clicked)
    back_button.on_click(on_back_clicked)
    delete_button.on_click(on_delete_clicked)


    # ウィジェットを表示
    sliders = widgets.VBox([low_h_slider, high_h_slider, low_s_slider, high_s_slider, low_v_slider, high_v_slider])
    display(sliders)
    display(widgets.HBox([image_widget, mask_image_widget]))
    display(widgets.HBox([play_button, stop_button, back_button, delete_button]))
    display(filename_label)



In [4]:
display_images_with_mask('../dataset/collect/', interval=0.5)

Label(value='0.98_115_3_44_cb64da28-d823-41cb-8fbb-5e01b1cd133b.png - 1/10')